In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
#import string

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

path_to_file = "summaries.txt"
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text = text.encode('ascii', errors = 'ignore')
text = text.decode('ascii')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))


Length of text: 1100289 characters


In [3]:
print(text[:250])

The European Patent Judges' Symposium (French: Colloque des juges europens de brevets, German: Symposium europischer Patentrichter) is a biennial symposium, with the claimed aim of providing a platform for national judges from legal systems with diff


In [4]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

90 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  '\r':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '$' :   7,
  '%' :   8,
  '&' :   9,
  "'" :  10,
  '(' :  11,
  ')' :  12,
  '*' :  13,
  '+' :  14,
  ',' :  15,
  '-' :  16,
  '.' :  17,
  '/' :  18,
  '0' :  19,
  ...
}


In [6]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'The European ' ---- characters mapped to int ---- > [53 70 67  3 38 83 80 77 78 67 63 76  3]


In [7]:
# The maximum length sentence we want for a single input in characters
seq_length = 100


examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

T
h
e
 
E


In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"The European Patent Judges' Symposium (French: Colloque des juges europens de brevets, German: Sympos"
'ium europischer Patentrichter) is a biennial symposium, with the claimed aim of providing a platform '
'for national judges from legal systems with differing traditions to exchange experiences and to there'
'by promote mutual understanding in the development of European patent law.Along with the Standing Adv'
'isory Committee before the European Patent Office (SACEPO), a committee advising the European Patent '


In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "The European Patent Judges' Symposium (French: Colloque des juges europens de brevets, German: Sympo"
Target data: "he European Patent Judges' Symposium (French: Colloque des juges europens de brevets, German: Sympos"


In [11]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 53 ('T')
  expected output: 70 ('h')
Step    1
  input: 70 ('h')
  expected output: 67 ('e')
Step    2
  input: 67 ('e')
  expected output: 3 (' ')
Step    3
  input: 3 (' ')
  expected output: 38 ('E')
Step    4
  input: 38 ('E')
  expected output: 83 ('u')


In [12]:
# Batch size 
BATCH_SIZE = 128
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of GRU units
rnn_units = 2048

In [14]:
import functools
rnn = functools.partial(
    tf.keras.layers.GRU,
    activation = 'relu', 
    recurrent_activation='sigmoid',
    kernel_initializer='glorot_uniform',
    recurrent_initializer='glorot_uniform',
    bias_initializer='zeros',
    dropout = 0.1,
    recurrent_dropout = 0.15)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,embedding_dim,
                                 batch_input_shape=[batch_size,None]),
        rnn(rnn_units,
           return_sequences = True,
           stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [30]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [31]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 90) # (batch_size, sequence_length, vocab_size)


In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (128, None, 256)          23040     
_________________________________________________________________
gru_2 (GRU)                  (128, None, 2048)         14161920  
_________________________________________________________________
dense_2 (Dense)              (128, None, 90)           184410    
Total params: 14,369,370
Trainable params: 14,369,370
Non-trainable params: 0
_________________________________________________________________


In [33]:
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) 
#sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [34]:
sampled_indices

array([67, 81, 19, 31, 78, 46, 26, 50, 56, 39, 36, 65, 40, 74, 73, 59, 23,
       83, 68, 66, 57, 59, 28, 77, 17, 65, 63, 26, 72, 78, 25, 27, 48, 83,
       57, 14, 27, 40, 70, 18, 22, 79, 56, 23, 78, 63, 81,  1, 78, 51, 31,
       60, 16, 86, 47, 30, 27, 57, 69, 35, 44, 52, 69, 29, 12,  6, 73, 61,
       80, 74, 80, 79, 73, 30, 41, 75,  1, 77, 73, 31, 79, 51, 65, 71, 40,
       87, 23, 36,  2, 79, 30, 39,  9, 15, 81, 69, 80, 37, 13, 46])

In [35]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'oet."  In his treatise, De perfecta poesi, he not only wrote that a poet "invents," "after a fashion'

Next Char Predictions: 
 'es0=pM7QWFCcGlkZ4ufdXZ9o.ca7jp68OuX+8Gh/3qW4pas\npR=[-xN;8XgBKSg:)#k]rlrqk;Hm\nok=qRciGy4C\rq;F&,sgrD*M'


In [36]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 100, 90)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.498675


In [37]:
model.compile(
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.0005, beta1= 0.8,
                                      beta2=0.9,epsilon = 1e-04),
    loss = loss)

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './text100'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [39]:
#22 seems to be the limit
EPOCHS=27

In [42]:
#'./text100/ckpt_27'
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [45]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/27
85/85 [==============================] - 74s 866ms/step - loss: 1.2264
Epoch 2/27
85/85 [==============================] - 75s 884ms/step - loss: 1.2181
Epoch 3/27
85/85 [==============================] - 75s 881ms/step - loss: 1.2118
Epoch 4/27
85/85 [==============================] - 74s 872ms/step - loss: 1.2025
Epoch 5/27
85/85 [==============================] - 75s 882ms/step - loss: 1.1963
Epoch 6/27
85/85 [==============================] - 76s 891ms/step - loss: 1.1885
Epoch 7/27
85/85 [==============================] - 80s 938ms/step - loss: 1.1827
Epoch 8/27
85/85 [==============================] - 73s 862ms/step - loss: 1.1754
Epoch 9/27
85/85 [==============================] - 75s 884ms/step - loss: 1.1678
Epoch 10/27
85/85 [==============================] - 77s 908ms/step - loss: 1.1610
Epoch 11/27
85/85 [==============================] - 73s 855ms/step - loss: 1.1543
Epoch 12/27
85/85 [==============================] - 73s 861ms/step - loss: 1.1480
Epoch 13/27
8

In [46]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text100/ckpt_27'

In [47]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            23040     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 2048)           14161920  
_________________________________________________________________
dense_3 (Dense)              (1, None, 90)             184410    
Total params: 14,369,370
Trainable params: 14,369,370
Non-trainable params: 0
_________________________________________________________________


In [49]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (''.join(text_generated))

In [51]:
# print(generate_text(model, start_string=u"ROMEO"))
#print(generate_text(model, start_string=u"uwu")[:300])
fileID = open("out.txt","w",encoding = "utf-8")
fileID.write(generate_text(model,start_string=u"Kapital"))
fileID.close()
